In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**데이터 분석**

In [ ]:
import numpy as np
import pandas as pd

data_path = '/kaggle/input/bike-sharing-demand/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

In [ ]:
train.shape, test.shape

# Q. CJW: 엥..왜 열 수가 이렇게 다르지?
# A. casual, registered, count가 train에 있는데..count를 예측할거다. --> casual, registered는 뺴야한다는군 (p.184)

In [ ]:
display(train.columns)
display(test.columns)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
'''
Data Fields
datetime - hourly date + timestamp  
season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
holiday - whether the day is considered a holiday
workingday - whether the day is neither a weekend nor holiday
weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
temp - temperature in Celsius
atemp - "feels like" temperature in Celsius
humidity - relative humidity
windspeed - wind speed
casual - number of non-registered user rentals initiated
registered - number of registered user rentals initiated
count - number of total rentals
'''

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission.head()

In [ ]:
# Q. CJW: p.185에서 datatime은 예측에 역할을 못하니까 뺀다고 하는데 진짜 필요없는지 확인해보자.
# 시간대별로 뭔가 써먹을 수 있는게 있지 않을까?
# A...아..연,월,일로 분리하기 때문에 필요없다는 의미구나..옥히

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
print(train['datetime'][1000])
print(train['datetime'][1000].split())
print(train['datetime'][1000].split()[0])
print(train['datetime'][1000].split()[1])

In [ ]:
print(train['datetime'][1000].split('-'))
print(train['datetime'][1000].split()[0].split('-'))
print(train['datetime'][1000].split()[0].split('-')[0])
print(train['datetime'][1000].split()[0].split('-')[1])
print(train['datetime'][1000].split()[0].split('-')[2])
print(train['datetime'][1000].split()[0].split('-')[-1])

In [ ]:
print(train['datetime'][1000].split(':'))
print(train['datetime'][1000].split()[1].split(':'))
print(train['datetime'][1000].split()[1].split(':')[0])
print(train['datetime'][1000].split()[1].split(':')[1])
print(train['datetime'][1000].split()[1].split(':')[2])

In [ ]:
type(train['datetime'][1000])

In [ ]:
print(train['datetime'][1000])
print(train['datetime'][1000].replace(':', '-').replace(' ', '-').split('-'))
#Q. CJW: 쓸데 없는 짓인듯 하다. 코드 가독이 더 떨어지는 듯한 느낌이군.

In [ ]:
train['date'] = train['datetime'].apply(lambda x: x.split()[0])

In [ ]:
train.date

In [ ]:
train['year'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[0])
train['month'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[1])
train['day'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[2])

train['hour'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[0])
train['minute'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[1])
train['second'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[2])

In [ ]:
train.tail()

In [ ]:
from datetime import datetime
import calendar

print(train['date'][1000])
print(datetime.strptime(train['date'][1000], '%Y-%m-%d'))

In [ ]:
print(datetime.strptime(train['date'][1000], '%Y-%m-%d').weekday())

In [ ]:
print(calendar.day_name[datetime.strptime(train['date'][1000], '%Y-%m-%d').weekday()])

In [ ]:
train['weekday'] = train['date'].apply(
    lambda dateString:
    calendar.day_name[datetime.strptime(dateString, "%Y-%m-%d").weekday()])

In [ ]:
train['weekday']

In [ ]:
train['season'] = train['season'].map(
    {
        1: 'Spring',
        2: 'Summer',
        3: 'Fall',
        4: 'Winter'
    })
train['weather'] = train['weather'].map(
    {
        1: 'Clear',
        2: 'Mist, Few clouds',
        3: 'Light Snow, Rain, Thunderstorm',
        4: 'Heavy Rain, Thunderstorm, Snow, Fog'
    })

In [ ]:
train.head()

In [ ]:
train.info()

**시각화**

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mpl.rc('font', size=15)
sns.displot(train['count'])

In [ ]:
sns.displot(data = train, x = 'count')
# 분포가 왼쪽으로 쏠려있다.. -> 정규 분포를 따르게 만들면 성능이 좋다는데 (p.193)
# 이러한 방법이 log변환이다.

In [ ]:
sns.displot(data = np.log(train['count']))

In [ ]:
mpl.rc('font', size = 14)
mpl.rc('axes', titlesize = 15)
figure, axes = plt.subplots(nrows = 3, ncols = 2)
plt.tight_layout()
figure.set_size_inches(10, 9)

sns.barplot(data = train, x = 'year', y = 'count', ax = axes[0, 0])
sns.barplot(data = train, x = 'month', y = 'count', ax = axes[0, 1])
sns.barplot(data = train, x = 'day', y = 'count', ax = axes[1, 0])
sns.barplot(data = train, x = 'hour', y = 'count', ax = axes[1, 1])
sns.barplot(data = train, x = 'minute', y = 'count', ax = axes[2, 0])
sns.barplot(data = train, x = 'second', y = 'count', ax = axes[2, 1])

axes[0, 0].set(title = 'Rental amounts by year')
axes[0, 1].set(title = 'Rental amounts by month')
axes[1, 0].set(title = 'Rental amounts by day')
axes[1, 1].set(title = 'Rental amounts by hour')
axes[2, 0].set(title = 'Rental amounts by minute')
axes[2, 1].set(title = 'Rental amounts by second')

axes[1, 0].tick_params(axis = 'x', labelrotation = 90)
axes[1, 1].tick_params(axis = 'x', labelrotation = 90)

In [ ]:
figure, axes = plt.subplots(nrows = 2, ncols = 2)
plt.tight_layout()
figure.set_size_inches(10, 10)

sns.boxplot(x = 'season', y = 'count', data = train, ax = axes[0, 0])
sns.boxplot(x = 'weather', y = 'count', data = train, ax = axes[0, 1])
sns.boxplot(x = 'holiday', y = 'count', data = train, ax = axes[1, 0])
sns.boxplot(x = 'workingday', y = 'count', data = train, ax = axes[1, 1])

axes[0, 0].set(title = "Box Plot On Count Across Season")
axes[0, 1].set(title = "Box Plot On Count Across Weather")
axes[1, 0].set(title = "Box Plot On Count Across Holiday")
axes[1, 1].set(title = "Box Plot On Count Across Working day")

axes[0, 1].tick_params(axis = 'x', labelrotation = 10)

#Q CJW: 이상치가 왜이리 많아???? 

In [ ]:
mpl.rc('font', size = 11)
figure, axes = plt.subplots(nrows = 5)
figure.set_size_inches(12, 18)

sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'workingday', ax = axes[0])
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'holiday', ax = axes[1])
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'weekday', ax = axes[2])
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'season', ax = axes[3])
sns.pointplot(x = 'hour', y = 'count', data = train, hue = 'weather', ax = axes[4])


In [ ]:
mpl.rc('font', size = 15)
figure, axes = plt.subplots(nrows = 2, ncols = 2)
plt.tight_layout()
figure.set_size_inches(7, 6)

sns.regplot(x = 'temp', y = 'count', data = train, ax = axes[0, 0],
           scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'blue'})
sns.regplot(x = 'atemp', y = 'count', data = train, ax = axes[0, 1],
           scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'blue'})
sns.regplot(x = 'windspeed', y = 'count', data = train, ax = axes[1, 0],
           scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'blue'})
sns.regplot(x = 'humidity', y = 'count', data = train, ax = axes[1, 1],
           scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'blue'})

In [ ]:
train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr()

In [ ]:
corrMat = train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr()
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
sns.heatmap(corrMat, annot = True)
ax.set(title = 'Heatmap of Numercal Data')

# 상관관계가 약한 피처는 제거한다. (p.208)

**모델**

In [ ]:
data_path

In [ ]:
#..ㅠㅠㅠ

import pandas as pd

data_path = '/kaggle/input/bike-sharing-demand/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

In [ ]:
train.head()

In [ ]:
train = train[train['weather'] != 4]

In [ ]:
#데이터 피쳐 조작 전에 train, test 합쳐서 한다.. 한꺼번에 처리하려고..
#test data에 없는 컬럼의 값은 NaN가 되는구나..
all_data = pd.concat([train, test], ignore_index = True)
all_data

In [ ]:
from datetime import datetime

all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])
#day는 의미 없다고 했다. train이랑 test가 날짜로 분리되서..분,초는 아까 다 0이었자나
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])
all_data['weekday'] = all_data['date'].apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').weekday())

In [ ]:
all_data

In [ ]:
drop_features = ['casual', 'registered', 'datetime', 'date', 'month', 'windspeed']

In [ ]:
all_data = all_data.drop(drop_features, axis = 1)

In [ ]:
all_data

In [ ]:
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

X_train = X_train.drop(['count'], axis = 1)
X_test = X_test.drop(['count'], axis = 1)

y = train['count']

In [ ]:
X_train.head()
#CJW..엥 season이랑 weather가 왜 아직 문자열이지???? -> 책에서는 train, test를 다시 읽었다.ㅠㅠ

**평가**

In [ ]:
import numpy as np

def rmsle(y_true, y_pred, convertExp = True):
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
        
    log_true = np.nan_to_num(np.log(y_true + 1))
    log_pred = np.nan_to_num(np.log(y_pred + 1))
    
    output = np.sqrt(np.mean((log_true - log_pred) ** 2))
    return output

In [ ]:
#basemodel
'''
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()
'''

In [ ]:
#basemodel
'''
log_y = np.log(y)
linear_reg_model.fit(X_train, log_y)
'''

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

randomforest_model = RandomForestRegressor()
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

rf_params = {'random_state':[42], 'n_estimators':[100, 120, 140]}
gridsearch_random_forest_model = GridSearchCV(estimator = randomforest_model,
                                             param_grid = rf_params,
                                             scoring=rmsle_scorer,
                                             cv=5)

log_y = np.log(y)
gridsearch_random_forest_model.fit(X_train, log_y)
print("최적 하이퍼파라미터:", gridsearch_random_forest_model.best_params_)

In [ ]:
#preds = linear_reg_model.predict(X_train)

In [ ]:
#print(f'linear regression RMSLE : {rmsle(log_y, preds, True):.4f}')

In [ ]:
#log에서 다시 복구하는걸 까먹지 말아야한다!!

'''
linearreg_preds = linear_reg_model.predict(X_test)

submission['count'] = np.exp(linearreg_preds)
submission.to_csv('submission.csv', index=False)
'''

In [ ]:
preds = gridsearch_random_forest_model.best_estimator_.predict(X_train)
print(f'랜덤 포레스트:{rmsle(log_y, preds, True):.4f}')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

randomforest_preds = gridsearch_random_forest_model.best_estimator_.predict(X_test)

figure, axes = plt.subplots(ncols = 2)
figure.set_size_inches(10, 4)

sns.histplot(y, bins = 50, ax = axes[0])
axes[0].set_title("Train Data Distribution")
sns.histplot(np.exp(randomforest_preds), bins = 50, ax = axes[1])
axes[1].set_title("Predicted Test Data Distribution")

In [ ]:
#log에서 다시 복구하는걸 까먹지 말아야한다!!
submission['count'] = np.exp(randomforest_preds)
submission.to_csv('submission.csv', index = False)